In [22]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, csc_matrix, find, coo_matrix
from scipy.sparse import eye, spdiags
from scipy.sparse.linalg import spsolve
from scipy.linalg import eigh
from scipy.io import  savemat
from joblib import Parallel, delayed

In [2]:
# Heat kernel graphs


def heat_kernel_on_graph(signal, laplacian, time, steps):
    h = time / steps
    n_vertices = laplacian.shape[0]

    # solve Ax=b
    time_step = lambda x: spsolve(eye(n_vertices) - h * laplacian, x)

    result = signal.copy()
    for i in range(steps):
        result = time_step(result)

        # help fix numerical issues
        result[result < 0] = 0
        result = result / np.sum(result) * np.sum(signal)

    return result


## Compute MESH heat kernel and conv wasserstein

In [36]:

def cot(x):
    return np.cos(x)/np.sin(x)


def cot_laplacian(X, T):
    nv = X.shape[0]
    nf = T.shape[0]

    # Find original edge lengths and angles
    L1 = np.linalg.norm(X[T[:, 1], :] - X[T[:, 2], :], axis=1)
    L2 = np.linalg.norm(X[T[:, 0], :] - X[T[:, 2], :], axis=1)
    L3 = np.linalg.norm(X[T[:, 0], :] - X[T[:, 1], :], axis=1)
    EL = np.column_stack((L1, L2, L3))
    A1 = (L2**2 + L3**2 - L1**2) / (2 * L2 * L3)
    A2 = (L1**2 + L3**2 - L2**2) / (2 * L1 * L3)
    A3 = (L1**2 + L2**2 - L3**2) / (2 * L1 * L2)
    A = np.column_stack((A1, A2, A3))
    A = np.arccos(A)

    # The Cot Laplacian
    I = [T[:, 0], T[:, 1], T[:, 2]]
    J = [T[:, 1], T[:, 2], T[:, 0]]
    S = 0.5 * 1 / np.tan([A[:, 2], A[:, 0], A[:, 1]]).flatten()
    
    In = np.vstack((I, J, I, J)).flatten()
    Jn = np.vstack((J, I, I, J)).flatten()
    Sn = np.vstack((-S, -S, S, S)).flatten()

    # Compute the areas. Use mixed weights Voronoi areas
    cA = 0.5 * 1 / np.tan(A)
    vp1 = [1, 2, 0]
    vp2 = [2, 0, 1]
    At = 1 / 4 * (EL[:, vp1] ** 2 * cA[:, vp1] + EL[:, vp2] ** 2 * cA[:, vp2])

    # Triangle areas
    N = np.cross(X[T[:, 0], :] - X[T[:, 1], :], X[T[:, 0], :] - X[T[:, 2], :])
    Ar = np.linalg.norm(N, axis=1)

    # Use barycentric area when cot is negative
    locs = np.where(cA[:, 0] < 0)[0]
    At[locs, 0] = Ar[locs] / 4
    At[locs, 1] = Ar[locs] / 8
    At[locs, 2] = Ar[locs] / 8
    locs = np.where(cA[:, 1] < 0)[0]
    At[locs, 0] = Ar[locs] / 8
    At[locs, 1] = Ar[locs] / 4
    At[locs, 2] = Ar[locs] / 8
    locs = np.where(cA[:, 2] < 0)[0]
    At[locs, 0] = Ar[locs] / 8
    At[locs, 1] = Ar[locs] / 8
    At[locs, 2] = Ar[locs] / 4

    # Vertex areas = sum triangles nearby
    I = np.vstack((T[:, 0], T[:, 1], T[:, 2])).flatten()
    J = np.ones(len(I))
    S = 0.5*cot(np.vstack((At[:, 0], At[:, 1], At[:, 2])).flatten())


    print("I",I)
    print("J",J)
    print("S",S)
    W = coo_matrix((Sn, (In, Jn)), shape=(nv, nv))
    A = coo_matrix((S, (I, J)), shape=(nv, 1))


    return W, A

def normv(V):
    return np.sqrt(np.sum(V**2, axis=1))

In [37]:


def get_mesh_data(X, T, num_eigs=10, name='mesh'):
    mesh = {}
    mesh['vertices'] = X
    mesh['triangles'] = T
    mesh['name'] = name
    
    mesh['cotLaplacian'], mesh['areaWeights'] = cot_laplacian(X, T)

    # Change to negative cot Laplacian and rescale to area = 1
    mesh['areaWeights'] = mesh['areaWeights'] / np.sum(mesh['areaWeights'])
    mesh['cotLaplacian'] = -1 * mesh['cotLaplacian']

    mesh['numVertices'] = X.shape[0]
    mesh['numTriangles'] = T.shape[0]

    evec = np.vstack((T[:, 0], T[:, 1], T[:, 1], T[:, 2], T[:, 0], T[:, 2])).T
    evec = np.unique(np.sort(evec, axis=1), axis=0)
    ordered_rows = np.where(evec[:, 0] < evec[:, 1])[0]
    mesh['edges'] = evec[ordered_rows, :]
    mesh['numEdges'] = mesh['edges'].shape[0]

    # Compute LB eigenstuff
    area_matrix = csr_matrix((mesh['areaWeights'], (np.arange(1, mesh['numVertices'] + 1), np.arange(1, mesh['numVertices'] + 1))))
    if num_eigs > 0:
        evals, evecs = eigh(mesh['cotLaplacian'].todense(), area_matrix.todense(), eigvals=(0, num_eigs - 1), overwrite_a=True, overwrite_b=True)
        mesh['laplaceBasis'] = evecs
        mesh['eigenvalues'] = evals

    normalf = np.cross(mesh['vertices'][mesh['triangles'][:, 1], :].T - mesh['vertices'][mesh['triangles'][:, 0], :].T,
                       mesh['vertices'][mesh['triangles'][:, 2], :].T - mesh['vertices'][mesh['triangles'][:, 0], :].T)
    d = np.sqrt(np.sum(normalf**2, axis=0))
    d[d < np.finfo(float).eps] = 1
    mesh['faceNormals'] = (normalf / d).T

    return mesh


In [38]:
# Heat kernel pour mesh


def heat_kernel_on_mesh(signal, M, time, steps, transpose=False):
    # equivalent de t pour time et steps pour m
    h = time / steps
    nv = M.numVertices

    # We really should pre-factor this for speed...
    blur_inverse = spdiags(M.areaWeights, 0, nv, nv) - h * M.cotLaplacian

    result = signal.copy()

    if not transpose:
        for i in range(steps):
            result = spsolve(blur_inverse, np.multiply(result, M.areaWeights))
    else:
        for i in range(steps):
            result = np.multiply(spsolve(blur_inverse.transpose(), result), M.areaWeights)

    return result


In [39]:
# Pour entropy du barycentre sur le chat





# Functions for mesh-related operations
def read_off(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        num_vertices, num_faces, _ = map(int, lines[1].split())
        vertices = np.array([list(map(float, line.split()[:3])) for line in lines[2:2 + num_vertices]])
        faces = np.array([list(map(int, line.split()[1:])) for line in lines[2 + num_vertices:]])
    return vertices, faces



# Main script
# Load shape
X, T = read_off('../data/wolf0.off')
M = get_mesh_data(X, T, 10)  # Compute 10 LB eigenfunctions for fun

# Design two distributions
blur_time = 0.0001
blur_steps = 10

front_vtx = [18962, 15966]
back_vtx = [26142, 22553]

p0 = np.zeros(M['numVertices'])
for i in range(len(front_vtx)):
    p0[front_vtx[i]] = 0.5 / M['areaWeights'][front_vtx[i]]
p0 = heat_kernel_on_mesh(p0, M, blur_time, blur_steps)

p1 = np.zeros(M['numVertices'])
for i in range(len(back_vtx)):
    p1[back_vtx[i]] = 0.5 / M['areaWeights'][back_vtx[i]]
p1 = heat_kernel_on_mesh(p1, M, blur_time, blur_steps)

# Set up Gaussian blur function for barycenter
blur_time = 0.00015
blur_steps = 10

blur = lambda x: heat_kernel_on_mesh(x, M, blur_time, blur_steps)
blur_transpose = blur

# Take the barycenter
p = np.column_stack((p0, p1))
p[p < 1e-10] = 1e-10
n_functions = 2

max_entropy = -np.sum(p * np.log(p) * np.tile(M['areaWeights'], (1, 2)), axis=0).max()

entropy_limits = [max_entropy, max_entropy + 1, max_entropy + 2, max_entropy + 3, np.inf]
n_entropies = len(entropy_limits)

euclidean_barycenter = np.sum(p, axis=1) / n_functions
alpha = np.array([1, 1])

options = {'niter': 100, 'unit_area_projection': 1}

barycenter = Parallel(n_jobs=-1)(
    delayed(convolutional_barycenter)(p, alpha, M['areaWeights'], blur, blur_transpose, entropy_limit, options)
    for entropy_limit in entropy_limits
)

savemat('entropyTest.mat', {'barycenter': barycenter, 'M': M, 'p0': p0, 'p1': p1})


I [array([   5,   21,   33, ..., 4332, 4282, 4334]), array([3108, 3085, 3197, ..., 4322, 4334, 4282]), array([3110, 3111,   34, ..., 4321, 4295, 4322])]
J [1. 1. 1.]
S [[1.57895614 0.31896955 0.55497207 ... 0.39536848 0.57965524 0.39036811]
 [1.57895614 0.60277205 0.55497207 ... 0.25946713 0.54219564 0.32249437]
 [0.71031185 0.33127019 0.05224948 ... 0.15021152 0.1161692  0.03779223]]


ValueError: row, column, and data arrays must be 1-D